# Quantum Computing Advanced - Assignment 2

# Made by Noeël (500787065) and Robin (500775219).

Welcome to the second assignment of the Quantum Computing Advanced course! For this assignment we will use Qiskit, a quantum computing framework developed by IBM.

## Qiskit environment

I assume you are familiar with Qiskit. Tutorials on how Qiskit works can be found in the official [Qiskit tutorials repository](https://github.com/Qiskit/qiskit-tutorials). The tutorials you might find useful in general are [Getting started with Qiskit](https://github.com/Qiskit/qiskit-tutorials/blob/master/qiskit/basics/getting_started_with_qiskit.ipynb) and [Summary of quantum operations](https://github.com/Qiskit/qiskit-tutorials/blob/master/qiskit/terra/summary_of_quantum_operations.ipynb). Besides from that you might want to consult the [Qiskit documentation](https://qiskit.org/documentation/) for some advanced details.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute
from qiskit import Aer
from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit.tools.visualization import plot_histogram, plot_state_city

import scipy as sp
import math
import itertools
import os
import secrets

# Defining the simulator you could use for this assignment
as_simulator = Aer.get_backend('qasm_simulator')

As mentioned during the lecture, the idea of this assignment is you to prepare a `possible` assignment for next year's students. Assume that the students will have the same knowledge that you had when the lecture began. You are required to elaborate `10` questions in total (obviously, you have to deliver the solution of your assignment).

Remember `all` the problems (not related to the content of the courses) that you faced in every single assignment you have solved while studying at HvA and try to learn from them. How would be the perfect assignment for a HvA HBO-ICT student?

The lecture included some RSA background that you could also include in the future assignment. It could be a great idea to prepare an assignment with all the steps of the process: key generation, message encryption (keep the original message for yourself and let's see if the student is able to decrypt it), hacking RSA with Shor's algorithm and message decryption.

If you decide to ask that type of question, you could use the following set of auxiliary routines. As mentioned during the lecture, you should understand them, add some description/documentation about these functions and finally add meaningful comments in the code. I juts copy and past them, so part of your work is to identify which is which.

A brief comment from the original assignment: `The routine genPandQ generates the primes p and q. The product p*q should be larger than nb-bits. The message should be less that p*q otherwise part of the message is lost by the modulo-operation.`

If you do not want to include RSA question in your assignment, then you can skip this part and go directly to the next cell.

In [2]:


def nbits(n):
    """
    Return the number of bits to represent `n`.
    :param n: int to represent in bits.
    :return: Int of how many bits are needed for given n.
    """
    return math.floor(math.log(n, 2))

def isprime(n):
    """
    Check if given is a prime number.
    :param n: an int.
    :return: Bool based on whether n is a prime number.
    """
    return len([ d for d in range(2, math.ceil(math.sqrt(n))+1) if n % d == 0]) == 0

def b2i(b):
    """
    Convert bytes to an int.
    :param b: Bytes.
    :return: int value of the bytes.
    """
    return int.from_bytes(b, 'little')

def i2b(i, n=1):
    """
    Convert int to Bytes.
    :param i: an int.
    :param n: ?
    :return: List of Bytes to represent the int.
    """
    return i.to_bytes(n, 'little')

def modinv(a, m):
    """
    calculate decryption key
    :param a: encryption key
    :param m: phi
    :return: decryption key
    """
    def egcd(a, b):
        """
        
        :param a: 
        :param b: 
        :return: 
        """
        if a == 0:
            return b, 0, 1
        else:
            g, y, x = egcd(b % a, a)
            return g, x - (b // a) * y, y

    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modinv: failure')
    else:
        return x % m

def genPandQ(nb):
    """
    Generate to prime numbers who product is less than `nb`-bits. 
    :param nb: Max product of the two prime numbers (N).
    :return: Two prime numbers (p an q)
    """
    lp = 2**((nb+2) // 2)
    while True:
        # To skip the small primes
        p = lp // 2 + secrets.randbelow(lp // 2)
        if not isprime(p):
            continue

        lq = 2**((nb+2) - nbits(p))
        while True:
            # To skip the small primes
            q = lq // 2 + secrets.randbelow(lq // 2)
            if not isprime(q): continue
            break
        break
    # Is n large enough to hold nb-bits
    if nbits(p*q) <= nb:
        print(p*q, p, q, nbits(p*q), nbits(p), nbits(q))
    assert nbits(p*q) > nb
    return p, q

If you plan to include only questions directly related to Shor's algorithm, then you should start from this cell.

As mentioned during the lecture, I found a couple of papers with some implementations of the Shor' algorithm. If you want to use them for your assignment, here is the list:

[Realization of a scalable Shor algorithm](https://science.sciencemag.org/content/351/6277/1068)  
[Experimental Demonstration of a Compiled Version of Shor’s Algorithm
with Quantum Entanglement](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.99.250505?casa_token=Ru-cllycgjMAAAAA%3Adowo43wJjc5gKFZ8pfHMXY2-L5B1knqgFl8zgZepYQQwrUVYdpjek-r_MM-CRTznnPgtuSCjyi0Z8WX4)  
[Simplified Factoring Algorithms for Validating Small-Scale Quantum Information Processing Technologies](https://arxiv.org/abs/1310.6446)  

The first paper is the one used in the slides of Leon and Frans (Quantum Computing Advanced from last year and Quantum Security from last block). The other two contain the circuits for the modular exponentiation that I included in my slides.

The following functions are used to: extract the counts from the simulation, compute the period of the function, compute the greatest common divisor (obtain `p` and `q`). As mentioned during the lecture, you should understand them, add some description/documentation about these functions and finally add meaningful comments in the code. I juts copy and past them, so part of your work is to identify which is which.

In [3]:
# def cf(n, d):
#     """
#     
#     :param n: 
#     :param d: 
#     :return: 
#     """
#     if d == 0:
#         return []
#     q = n // d
#     r = n % d
#     return [q] + cf(d, r)
# 
# def cf2r(seq):
#     """
#     
#     :param seq: 
#     :return: 
#     """
#     num, den = 1, 0
#     for u in reversed(seq):
#         num, den = den + num*u, num
#     return num, den
# 
# def per(n, d):
#     """
#     
#     :param n: 
#     :param d: 
#     :return: 
#     """
#     _cf = cf(n, d)
#     ps = []
#     for i in range(1, len(_cf)+1):
#         dn = cf2r(_cf[:i])
#         ps.append(('{}/{}'.format(dn[0], dn[1]), dn[1]))
#     return ps
# 
# ps = set()
# mean = sum(counts.values())/len(counts.values())
# 
# for vec in sorted(counts.keys()):
#     x = int(vec, 2)
#     if counts[vec] < mean: continue
#     _ps = per(x, a**(4))
#     ps.update([ nd[1] for nd in _ps ])
# print('ps={}'.format(sorted(ps)))
# 
# ps = [ p for p in ps if p > 1 and p % 2 == 0 and a**p % N == 1 ]
# print('ps={}'.format(sorted(ps)))
# 
# for p in sorted(ps):
#     f1 = math.gcd(a**(p//2)-1, N)
#     f2 = math.gcd(a**(p//2)+1, N)
#     if f1 == 1 or f2 == 1: continue
#     print('a=', a)
#     print('p=', p)
#     print('e= {} # gcd({}-1, {})'.format(f1, a**(p//2), N))
#     print('d= {} # gcd({}+1, {})'.format(f2, a**(p//2), N))
#     print()
#     

## Assignment 1

First some questions about RSA. To setup RSA for crack.

RSA Key generation.

Calculate `N` and `phi`, given `p` and `q`. 

In [4]:
p = 29
q = 31

# Answer
N = p * q
phi = (p-1) * (q-1)

## Assignment 2

Calculate `e` (encryption) and `d` (decryption) key with `N` and `Phi` from the previous anwser.  

In [5]:
# Answer
from math import gcd

comprimes = []
for i in range(2, phi):
     if (gcd(i, phi) == 1): # if coprime with phi
         if(gcd(i, N) == 1): # and coprime with N
             comprimes.append(i)
             
print(comprimes)
# we'll just pick the first one
e = comprimes[0]

d = modinv(int(e), int(phi))

print("e: {}, d: {}".format(e, d))

[11, 13, 17, 19, 23, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 121, 127, 131, 137, 139, 143, 149, 151, 157, 163, 167, 169, 173, 179, 181, 187, 191, 193, 197, 199, 209, 211, 221, 223, 227, 229, 233, 239, 241, 247, 251, 253, 257, 263, 269, 271, 277, 281, 283, 289, 293, 299, 307, 311, 313, 317, 323, 331, 337, 347, 349, 353, 359, 361, 367, 373, 379, 383, 389, 391, 397, 401, 407, 409, 419, 421, 431, 433, 437, 439, 443, 449, 451, 457, 461, 463, 467, 473, 479, 481, 487, 491, 499, 503, 509, 517, 521, 523, 529, 533, 541, 547, 557, 559, 563, 569, 571, 577, 583, 587, 593, 599, 601, 607, 611, 613, 617, 619, 629, 631, 641, 643, 647, 649, 653, 659, 661, 671, 673, 677, 683, 689, 691, 697, 701, 703, 709, 719, 727, 731, 733, 737, 739, 743, 751, 757, 761, 767, 769, 773, 779, 781, 787, 793, 797, 799, 803, 809, 811, 817, 821, 823, 827, 829, 839]
e: 11, d: 611


## Assignment 3


RSA message encryption.

Please encrypt the message (`me`) using your public key (`e`, `N`)


In [6]:
me = 2

# Answer
import math

c = int(me ** e) % N
print(c)

250


## Assignment 4

RSA message decryption.

Now using your private key (`d`, `N`), decrypt the message from assignment 3. 

In [7]:
# Answer
# import math
md = int(c ** d) % N
print(md)

2


## Assignment 5

Now encrypt a message using your private key (`d`) en decrypt it using your public key (`e`). 
What is the result? And why is this usefull?

In [8]:
# Answer
import math
message_blank = 12
message_encypted = int(message_blank ** d) % N
message_decrypted = int(message_encypted ** e) % N
print(message_blank)
print(message_decrypted)

# this is usefull for message signing!

12
12


## Assignment 6
Now we will crack our PKI (Public Key Infrastructure).
The first step is to choice `a`. `a` must be  `a < N` and coprime.

In [9]:
# Answer
import random
coprimes_a = []
for i in range(3, N):
     if (gcd(i, N) == 1):
             coprimes_a.append(i)

a = coprimes_a[random.randint(0, len(coprimes_a)-1)]
print(a)

688


## Assignment 7

Determine the unknown peroid `r` using `a`. `r` is the peroid of the function `a^x (mod N)` 

In [10]:
# Answer
import math
found_r = False
i = 0
# does not work if one number apprears multiple times in a period.
first_try = (a ** i) % N 
candidate_r = first_try
while not found_r:
    i+=1
    candidate_r = (a ** i) % N
    if candidate_r == first_try:
        r = i
        found_r = True

print(r)

84


## Assignment 8

Check if `r` is correct for futher calculations.

In [11]:
# Answer
import math

check_even = (((a ** int(r/2)) - 1) * ((a ** int(r/2)) + 1)) % N
if (check_even):
    print("not even, start over!")
    
if not ((a ** int(r/2)) + 1) % N:
    print("zero, start over!")

## Assignment 9

compute `p` and `q` 

In [12]:
# Answer

import math

p = gcd(int(a ** int(r/2)) - 1, N)
p = gcd(int(a ** int(r/2)) +1, N)
print("p: {}, q: {}".format(p, q))

p: 29, q: 31


## Assignment 10

Now calculate `r` using Qiskit.

In [13]:
# Answer

# something for you to work on during the holidays ;) https://science.sciencemag.org/content/351/6277/1068